In [1]:
from pathlib import Path

import automech as am
import polars as pl
from project_utilities import p_

open_browser = False
tag = "Z_mess_v0"
root_path = Path("../..")

stoich = "C5H9O"

calc_mech_json = p_.calculated_pes_mechanism(
    tag, stoich, "json", path=p_.data(root_path)
)
calc_mech = am.io.read(calc_mech_json)

if open_browser:
    disp_mech = calc_mech.model_copy(deep=True)
    disp_mech.reactions = disp_mech.reactions.filter(~pl.col("well_skipping"))
    am.display(disp_mech)

In [2]:
import altair as alt
import autochem as ac
import numpy as np
from project_utilities import fig

reactants = ("C5H8(522)", "OH(4)")
rate_df = fig.rate_data(calc_mech, reactants, min_branch_frac=0.01)

branch_frac_chart = fig.branching_fraction_chart(
    rate_df, P_range=(0.1, 100), T=825, legend=False, total=True
)
rate_chart = fig.rate_chart(
    rate_df,
    T_range=(500, 1200),
    P=1,
    legend=False,
    total=True,
    units={"substance": "molecules"},
)
chart = alt.vconcat(
    branch_frac_chart.properties(width=300, height=150),
    rate_chart.properties(width=300, height=240),
).configure_axis(grid=False)
chart

alt.VConcatChart(...)

In [3]:
from importlib import reload

from autochem.util import plot

reload(plot)

expt = ac.rate.data.Rate(
    order=2,
    k_data=np.array(
        [
            [2.41e-11],
            [2.99e-11],
            [3.65e-11],
            [4.49e-11],
            [3.62e-11],
            [3.47e-11],
            [3.84e-11],
            [3.16e-11],
        ]
    ),
    T=[856, 978, 1076, 1258, 1109, 1107, 1148, 1025],
    P=[1],
    units={"substance": "molec"},
)

T, k = expt.plot_data(T=(500, 1250), P=1, units={"substance": "molecules"})
expt_chart = plot.general(
    y_data=[k],
    x_data=T,
    labels=["Liu"],
    colors=[plot.Color.black],
    mark="point",
    mark_kwargs={"opacity": 1, "filled": True, "shape": "diamond", "size": 80},
    y_scale=plot.regular_scale((np.min(k), np.max(k))),
    legend=False,
)
rate_chart_with_expt = alt.layer(rate_chart, expt_chart).resolve_scale(
    color="independent"
)

chart = alt.vconcat(
    branch_frac_chart.properties(width=300, height=150),
    rate_chart_with_expt.properties(width=300, height=240),
).configure_axis(grid=False)
chart

alt.VConcatChart(...)